In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
# import warnings
# warnings.filterwarnings('ignore')
#Nnormal
#supraventricualr
#venticuar trchicardia
#fusion
#Q
# load data
train_df=pd.read_csv('dn.csv',header=None)
test_df=pd.read_csv('dt.csv',header=None)
train_df.tail()

c:\Users\ghanshyam\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
87549,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87550,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87551,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87552,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87553,0.901506,0.845886,0.800695,0.748552,0.687138,0.599073,0.512167,0.427578,0.395133,0.402086,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [2]:
train_df[187]=train_df[187].astype(int)
train_df[187].value_counts()

0    72471
4     6431
2     5788
1     2223
3      641
Name: 187, dtype: int64

In [3]:
test_df[187]=test_df[187].astype(int)
test_df[187].value_counts()

0    18118
4     1608
2     1448
1      556
3      162
Name: 187, dtype: int64

In [4]:
target_train=train_df[187]
target_test=test_df[187]
y_train=to_categorical(target_train)
y_test=to_categorical(target_test)
X_train=train_df.iloc[:,:186].values
X_test=test_df.iloc[:,:186].values
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)
# target_train
# y_train
X_train.shape

(87554, 186, 1)

In [6]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
def network(X_train,y_train,X_test,y_test):
    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (3), activation='relu')(pool1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu')(pool2)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    # lstm1=LSTM(100)(pool3)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(5, activation='softmax', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
    
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=10,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))
    model.load_weights('best_model.h5')
    return(model,history)

In [7]:

model,history=network(X_train,y_train,X_test,y_test)

Epoch 1/10
2737/2737 [==============================] - 41s 14ms/step - loss: 0.1868 - accuracy: 0.9484 - val_loss: 0.1136 - val_accuracy: 0.9687
Epoch 2/10
2737/2737 [==============================] - 37s 14ms/step - loss: 0.0924 - accuracy: 0.9737 - val_loss: 0.0945 - val_accuracy: 0.9742
Epoch 3/10
2737/2737 [==============================] - 37s 13ms/step - loss: 0.0715 - accuracy: 0.9792 - val_loss: 0.0859 - val_accuracy: 0.9769
Epoch 4/10
2737/2737 [==============================] - 35s 13ms/step - loss: 0.0606 - accuracy: 0.9818 - val_loss: 0.0758 - val_accuracy: 0.9793
Epoch 5/10
2737/2737 [==============================] - 37s 13ms/step - loss: 0.0526 - accuracy: 0.9845 - val_loss: 0.0774 - val_accuracy: 0.9792
Epoch 6/10
2737/2737 [==============================] - 36s 13ms/step - loss: 0.0482 - accuracy: 0.9854 - val_loss: 0.0689 - val_accuracy: 0.9828
Epoch 7/10
2737/2737 [==============================] - 37s 14ms/step - loss: 0.0417 - accuracy: 0.9871 - val_loss: 0.0694 -

In [8]:
model.evaluate((X_test),y_test, verbose=0)

[0.06481361389160156, 0.9840580821037292]

In [9]:
y_valid=target_test
yf=model.predict(X_test)
# san
nrow=len(yf)
nrow
ncol=len(yf[0])
ncol

685/685 [==============================] - 4s 6ms/step


5

In [11]:
y_pred=[]
for i in range(nrow):
        max1 = 0
        p=-1
        for j in range(ncol):
            if yf[i][j] > max1 :
                max1 = yf[i][j]
                p=j
                 
        y_pred.append(p)

In [12]:
y_true=target_test.to_numpy()

y_pred
xxx=pd.Series(y_pred)
xxx.value_counts()

0    18261
4     1584
2     1422
1      486
3      139
dtype: int64

In [13]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(y_true, y_pred)
cf_matrix

array([[18041,    51,    19,     3,     4],
       [  111,   430,    14,     0,     1],
       [   57,     4,  1372,    12,     3],
       [   25,     0,    13,   124,     0],
       [   27,     1,     4,     0,  1576]], dtype=int64)

In [14]:
test=pd.read_csv('testcase.csv')
# model.predict()
test=test.to_numpy()
test=test.reshape(1,186,1)
model.predict(test)
# test

1/1 [==============================] - 0s 26ms/step


array([[9.99999642e-01, 1.14210756e-07, 4.54932425e-10, 2.28934340e-07,
        6.64396871e-10]], dtype=float32)